In [ ]:
import os

import keras
from keras.layers import Input, Concatenate
from keras.models import Model, load_model

In [ ]:
model_dir = '../models/'
model_names = [
    '48px_v3.2_2024-01-07.h5', 
    '48px_v3.3_2024-01-08.h5',
    '48px_v3.4_2024-01-08.h5',
    '48px_v3.5_2024-01-08.h5',
    '48px_v3.6_2024-01-10.h5', 
    '48px_v3.7_2024-01-10.h5']
    
models = []
for name in model_names:
    model = load_model(os.path.join(model_dir, name))
    model._name = name[:9]
    models.append(model)

In [ ]:
for model in models:
    print(model.summary())

In [ ]:
input_layer = Input(shape=models[0].input_shape[1:]) 

In [ ]:
for model in models:
    assert input_layer.shape == model.input_shape

In [ ]:
model_outputs = []
for model in models:  
    model_output = model(input_layer)
    model_outputs.append(model_output)

concatenated_output = Concatenate()(model_outputs)
ensemble_model = Model(input_layer, concatenated_output)

ensemble_model.compile(
        optimizer=keras.optimizers.Adam(3e-4),
        loss=keras.losses.BinaryCrossentropy(from_logits=False),
        metrics=[keras.metrics.BinaryAccuracy(name="acc")],
    )
ensemble_model._name = 'v3.2-3.7ensemble'
ensemble_model.summary()

In [ ]:
ensemble_model.save(os.path.join(model_dir, '48px_v3.2-3.7ensemble_2024-02-13.h5'))